# Introduction

`pyiron_ontology` uses the `owlready2` library to build up pyiron-specific ontologies, and provides some extra tools to help you leverage these.

At present, the only ontology implemented is for the realm of atomistic calculations, and the scope of this ontology is still fairly limited.

First, let's import the ontology (an `owlready2.namespace.Ontology` object we define in the `pyiron_atomistics` repository on GitHub

In [1]:
from pyiron_ontology import atomistics_onto as onto

We can look at various properties of the ontology, just like other owl ontologies, e.g. the classes and individuals defined in this space:

In [5]:
list(onto.classes())

[pyiron.PyObject,
 pyiron.Parameter,
 pyiron.InputParameter,
 pyiron.OutputParameter,
 pyiron.GenericParameter,
 pyiron.Code,
 pyiron.Label]

In [6]:
list(onto.individuals())

[pyiron.Atomistic,
 pyiron.lCode,
 pyiron.DFT,
 pyiron.MaterialProperty,
 pyiron.PeriodicBoundaryConditions,
 pyiron.UserInput,
 pyiron.Bulk3dStructure,
 pyiron.AtomisticEnergyCalculator,
 pyiron.ChemicalElement,
 pyiron.AtomicStructure,
 pyiron.Executable,
 pyiron.CreateStructureBulk,
 pyiron.CreateStructureBulk/input/element,
 pyiron.CreateStructureBulk/output/structure,
 pyiron.CreateSurface,
 pyiron.CreateSurface/input/element,
 pyiron.CreateSurface/output/structure,
 pyiron.Bulk_modulus,
 pyiron.B_prime,
 pyiron.Murnaghan,
 pyiron.Murnaghan/output/equilibrium_bulk_modulus,
 pyiron.Murnaghan/output/equilibrium_b_prime,
 pyiron.Murnaghan/ref_job,
 pyiron.EnergyCutoff,
 pyiron.VASP,
 pyiron.ENCUT,
 pyiron.IBRAV,
 pyiron.VASP/input/structure,
 pyiron.ETOT,
 pyiron.LAMMPS,
 pyiron.LAMMPS/input/structure]

We can make the usual owlready queries of these objects, e.g.

In [8]:
onto.VASP.domain

[pyiron.Atomistic, pyiron.lCode, pyiron.DFT]

In [7]:
onto.VASP.is_a

[pyiron.Code,
 pyiron.has_conditions.only(owl.Nothing),
 pyiron.has_transitive_conditions.only(owl.Nothing),
 pyiron.has_options.only(OneOf([pyiron.Bulk3dStructure, pyiron.AtomisticEnergyCalculator])),
 pyiron.is_in_domains.only(OneOf([pyiron.Atomistic, pyiron.lCode, pyiron.DFT])),
 pyiron.has_generic_parameter.only(OneOf([pyiron.Executable])),
 pyiron.has_input.only(OneOf([pyiron.ENCUT, pyiron.IBRAV])),
 pyiron.has_mandatory_input.only(OneOf([pyiron.VASP/input/structure])),
 pyiron.has_output.only(owl.Nothing)]

We can also look into some of the atomistics-specific relationships that have been defined:

In [10]:
onto.VASP.mandatory_input

[pyiron.VASP/input/structure]

and we can chain these queries together in meaningful ways:

In [25]:
some_code = onto.VASP
first_input = some_code.mandatory_input[0]
appears_elsewhere = first_input.generic_parameter[0].has_parameters
can_come_from = first_input.consistent_output(first_input.has_transitive_conditions)
which_is_produced_by = can_come_from[0].output_of
print('some_code', some_code)
print('first_input', first_input)
print('appears_elsewhere', appears_elsewhere)
print('can_come_from', can_come_from)
print('which_is_produced_by', which_is_produced_by)

some_code pyiron.VASP
first_input pyiron.VASP/input/structure
appears_elsewhere [pyiron.CreateStructureBulk/output/structure, pyiron.CreateSurface/output/structure, pyiron.VASP/input/structure, pyiron.LAMMPS/input/structure]
can_come_from [pyiron.CreateStructureBulk/output/structure]
which_is_produced_by [pyiron.CreateStructureBulk]


This is powerful, but can be a bit unwieldly. 
`pyiron_ontology` also comes back with a special helper class for performing reasoning that is specific to this ontology and makes it easier to perform some helpful ontological queries and pyiron database and storage interactions.
Let's import that and use it.

We'll also import `pyiron_atomistics.Project` so we can create some data to work with.

In [26]:
from pyiron_ontology import AtomisticsReasoner
from pyiron_atomistics import Project
import numpy as np

In [27]:
reasoner = AtomisticsReasoner(onto) 

First, we'll use the reasoner to automatically build a tree of possible workflows that all end in the generation of a particular parameter:

In [28]:
bulk_modulus_worflows = reasoner.build_tree(onto.Bulk_modulus)
bulk_modulus_worflows.render()

(Target, Executable, Req. Inputs) (note: Target == None --> Use the Executable as input)

(pyiron.Murnaghan/output/equilibrium_bulk_modulus, pyiron.Murnaghan, [pyiron.Murnaghan/ref_job])
	(None, pyiron.VASP, [pyiron.VASP/input/structure])
		(pyiron.CreateStructureBulk/output/structure, pyiron.CreateStructureBulk, [pyiron.CreateStructureBulk/input/element])
			User input
	(None, pyiron.LAMMPS, [pyiron.LAMMPS/input/structure])
		(pyiron.CreateStructureBulk/output/structure, pyiron.CreateStructureBulk, [pyiron.CreateStructureBulk/input/element])
			User input


Next, we'll produce some data and then use the a tool on the reasoner to search over it for data that matches a particular ontological property.

First, we'll need to produce some data to search over. In this case, let's calculate the bulk modulus for a couple of alloys with varying Nickle content. On a single-core laptop, this might take two or three minutes.

In [29]:
pr = Project('example')
pr.remove_jobs(recursive=True)

Are you sure you want to delete all jobs from 'example'? y/(n) y


In [30]:
for host in ['Al', 'Cu']:
    for frac in [0., 0.10, 0.25]:
        ref = pr.atomistics.job.Lammps(f"Lammps_host{host}_frac{frac:.2f}".replace(".", "d"))
        ref.structure = pr.atomistics.structure.bulk(host, cubic=True).repeat(3)
        random_ids = np.random.choice(
            np.arange(len(ref.structure), dtype=int), 
            int(frac * len(ref.structure))
        )
        ref.structure[random_ids] = "Ni"
        ref.potential = ref.list_potentials()[0]

        murn = pr.atomistics.job.Murnaghan(f"Murn_host{host}_frac{frac:.2f}".replace(".", "d"))
        murn.input['num_points']=7
        murn.ref_job = ref
        murn.run()

The job Murn_hostAl_frac0d00 was saved and received the ID: 274
The job Murn_hostAl_frac0d00_0_9 was saved and received the ID: 275
The job Murn_hostAl_frac0d00_0_9333333 was saved and received the ID: 276
The job Murn_hostAl_frac0d00_0_9666667 was saved and received the ID: 277
The job Murn_hostAl_frac0d00_1_0 was saved and received the ID: 278
The job Murn_hostAl_frac0d00_1_0333333 was saved and received the ID: 279
The job Murn_hostAl_frac0d00_1_0666667 was saved and received the ID: 280
The job Murn_hostAl_frac0d00_1_1 was saved and received the ID: 281
The job Murn_hostAl_frac0d10 was saved and received the ID: 282
The job Murn_hostAl_frac0d10_0_9 was saved and received the ID: 283
The job Murn_hostAl_frac0d10_0_9333333 was saved and received the ID: 284
The job Murn_hostAl_frac0d10_0_9666667 was saved and received the ID: 285
The job Murn_hostAl_frac0d10_1_0 was saved and received the ID: 286
The job Murn_hostAl_frac0d10_1_0333333 was saved and received the ID: 287
The job Murn_h

2023-01-20 15:28:33,975 - pyiron_log - WARNING - Minimum could not be found!


The job Murn_hostCu_frac0d00 was saved and received the ID: 298
The job Murn_hostCu_frac0d00_0_9 was saved and received the ID: 299
The job Murn_hostCu_frac0d00_0_9333333 was saved and received the ID: 300
The job Murn_hostCu_frac0d00_0_9666667 was saved and received the ID: 301
The job Murn_hostCu_frac0d00_1_0 was saved and received the ID: 302
The job Murn_hostCu_frac0d00_1_0333333 was saved and received the ID: 303
The job Murn_hostCu_frac0d00_1_0666667 was saved and received the ID: 304
The job Murn_hostCu_frac0d00_1_1 was saved and received the ID: 305
The job Murn_hostCu_frac0d10 was saved and received the ID: 306
The job Murn_hostCu_frac0d10_0_9 was saved and received the ID: 307
The job Murn_hostCu_frac0d10_0_9333333 was saved and received the ID: 308
The job Murn_hostCu_frac0d10_0_9666667 was saved and received the ID: 309
The job Murn_hostCu_frac0d10_1_0 was saved and received the ID: 310
The job Murn_hostCu_frac0d10_1_0333333 was saved and received the ID: 311
The job Murn_h

Now let's search the pyiron database for instances of some of our physically-meaningful properties:

In [31]:
reasoner.search_database_for_property(onto.Bulk_modulus, pr)

,Chemical Formula,Bulk_modulus [MPa],Engine
0,Al108,81146.527225,Lammps
1,Al98Ni10,85981.780141,Lammps
2,Al86Ni22,NaN,Lammps
3,Cu108,141955.804050,Lammps
4,Cu99Ni9,146404.417486,Lammps
5,Cu84Ni24,150269.234954,Lammps


We can also filter our search by chemistry:

In [32]:
reasoner.search_database_for_property(onto.B_prime, pr, select_alloy="Cu")

,Chemical Formula,B_prime [1],Engine
0,Cu108,4.394886,Lammps
1,Cu99Ni9,4.497053,Lammps
2,Cu84Ni24,3.649895,Lammps


# Cleanup

In [33]:
pr.remove_jobs_silently(recursive=True)
pr.remove(enable=True)

<ipython-input-33-18562755bd1e>:1: DeprecationWarning: pyiron_base.project.generic.remove_jobs_silently is deprecated: Use pr.remove_jobs(silently=True) rather than pr.remove_jobs_silently()..
  pr.remove_jobs_silently(recursive=True)


  0%|          | 0/48 [00:00<?, ?it/s]